In [1]:
import json

import pandas as pd

import autogen
from autogen import AssistantAgent, UserProxyAgent

c:\TFS\Study\agentic-ai\.venv\lib\site-packages\flaml\__init__.py:20: UserWarning: flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.
  warnings.warn("flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.")


In [2]:
config_list = autogen.config_list_from_json(
    env_or_file="AOAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": {
            "gpt-4o"
        }
    },
)

In [3]:
# Start logging
logging_session_id = autogen.runtime_logging.start(config={"dbname": "logs.db"})
print("Logging session ID: " + str(logging_session_id))


Logging session ID: caed33a2-82e9-46ff-ac41-e07684ae80b9


In [4]:
# Create an agent workflow and run it
assistant = AssistantAgent(name="assistant", llm_config= {"config_list": config_list})
user_proxy = UserProxyAgent(
    name="user_proxy",
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
)

In [5]:
user_proxy.initiate_chat(
    assistant, message="What is the height of the Eiffel Tower? Only respond with the answer and terminate"
)

user_proxy (to assistant):

What is the height of the Eiffel Tower? Only respond with the answer and terminate

--------------------------------------------------------------------------------
assistant (to user_proxy):

The height of the Eiffel Tower is 330 meters. TERMINATE

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'What is the height of the Eiffel Tower? Only respond with the answer and terminate', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'The height of the Eiffel Tower is 330 meters. TERMINATE', 'role': 'user', 'name': 'assistant'}], summary='The height of the Eiffel Tower is 330 meters. ', cost={'usage_including_cached_inference': {'total_cost': 0.002605, 'gpt-4o-2024-05-13': {'cost': 0.002605, 'prompt_tokens': 482, 'completion_tokens': 13, 'total_tokens': 495}}, 'usage_excluding_cached_inference': {'total_cost': 0.002605, 'gpt-4o-2024-05-13': {'cost': 0.002605, 'prompt_tokens': 482, 'completion_tokens': 13, 'total_tokens': 495}}}, human_input=[])

In [6]:
autogen.runtime_logging.stop()

Getting Data from the SQLite Database

In [7]:
def get_log(dbname="logs.db", table="chat_completions"):
    import sqlite3

    con = sqlite3.connect(dbname)
    query = f"SELECT * from {table}"
    cursor = con.execute(query)
    rows = cursor.fetchall()
    column_names = [description[0] for description in cursor.description]
    data = [dict(zip(column_names, row)) for row in rows]
    con.close()
    return data

In [8]:
def str_to_dict(s):
    return json.loads(s)


log_data = get_log()
log_data_df = pd.DataFrame(log_data)

log_data_df["total_tokens"] = log_data_df.apply(
    lambda row: str_to_dict(row["response"])["usage"]["total_tokens"], axis=1
)

log_data_df["request"] = log_data_df.apply(lambda row: str_to_dict(row["request"])["messages"][0]["content"], axis=1)

log_data_df["response"] = log_data_df.apply(
    lambda row: str_to_dict(row["response"])["choices"][0]["message"]["content"], axis=1
)

log_data_df

,id,invocation_id,client_id,wrapper_id,session_id,source_name,request,response,is_cached,cost,start_time,end_time,total_tokens
0,1,aa5b69b6-ee76-42f8-9292-fb9d4adff22d,2222156761744,2222156766160,caed33a2-82e9-46ff-ac41-e07684ae80b9,assistant,You are a helpful AI assistant.\nSolve tasks u...,The height of the Eiffel Tower is 330 meters. ...,0,0.002605,2024-12-29 16:00:17.908613,2024-12-29 16:00:19.224969,495


Computing Cost

In [9]:
# Sum totoal tokens for all sessions
total_tokens = log_data_df["total_tokens"].sum()

# Sum total cost for all sessions
total_cost = log_data_df["cost"].sum()

# Total tokens for specific session
session_tokens = log_data_df[log_data_df["session_id"] == logging_session_id]["total_tokens"].sum()
session_cost = log_data_df[log_data_df["session_id"] == logging_session_id]["cost"].sum()

print("Total tokens for all sessions: " + str(total_tokens) + ", total cost: " + str(round(total_cost, 4)))
print(
    "Total tokens for session "
    + str(logging_session_id)
    + ": "
    + str(session_tokens)
    + ", cost: "
    + str(round(session_cost, 4))
)

Total tokens for all sessions: 495, total cost: 0.0026
Total tokens for session caed33a2-82e9-46ff-ac41-e07684ae80b9: 495, cost: 0.0026
